In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

#### Aquisição dos dados

In [2]:
# Importar dados de arquivo csv, onde o separador de campos é '.' e o separador de casas decimais é ','
df = pd.read_csv('dados/Indices_28_dias/dados_isolador40.csv', sep=';', decimal=',')

FileNotFoundError: [Errno 2] File b'dados/Indices_28_dias/dados_isolador40.csv' does not exist: b'dados/Indices_28_dias/dados_isolador40.csv'

#### Análise exploratória dos dados

In [ ]:
# 5 primeiros registros do dataframe
df.head()

In [ ]:
# Formato do dataframe (registros/colunas)
df.shape

In [ ]:
# Informações estatísticas do dataframe
df.describe()
# Notamos que existem atributos com valor mínimo igual a zero.

In [ ]:
# Verificando se existem valores nulos/missing
df.isnull().values.any()

In [ ]:
# Verificando a quantidade de registros nulos
df.isnull().sum()

In [ ]:
# Verificando a quantidade de registros com valor zero em cada atributo.
(df == 0).sum()
# Devido à quantidade de valores zerados no atributo chuva, em relação ao total de registros,
# verificar se o atributo exerce alguma influência no modelo.

In [ ]:
# Plotando a correlação entre os atributos
df.corr().style.format("{:.5}").background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)

In [ ]:
# Correlação da corrente de fuga com os demais atributos.
df.corr()['corrente_fuga']
# Verificamos que a pressão possui uma correlação muito baixa, verificar se faz alguma diferença no modelo.

In [ ]:
plt.figure(figsize=(18,8))
sns.set_style('darkgrid')
sns.distplot(df.corrente_fuga);
# A distribuição da corrente de fuga segue uma distribuição normal.

#### Preparação dos dados para o modelo

Apesar do atributo chuva possuir muitos registros com valor zero, e da baixa correlação da pressão, os dois atributos serão mantidos para a criação do modelo base.

In [ ]:
# Excluindo a variável target 'corrente_fuga'.
cols = df.columns.drop(['corrente_fuga'])
cols

In [ ]:
# 80% dos dados para treino e 20% para teste.
# O parâmetro 'random_state' (semente aleatória) garante que os resultados sejam repetíveis em qualquer máquina.
X_train, X_test, y_train, y_test = train_test_split(df[cols], df['corrente_fuga'], test_size=0.20, random_state=42)

#### Criando os modelos de machine learning

##### 01. Regressão Linear

In [ ]:
lr = LinearRegression()

In [ ]:
# Treinando o modelo.
model_lr = lr.fit(X_train, y_train)
# Realizando a predição com os dados de teste.
result_lr = model_lr.predict(X_test)

Calculando o erro e computando o score para os dados de teste:

In [ ]:
# MSE: média do erro quadrado. Quanto menor o valor, mais acertivo é o modelo.
mse_lr = mean_squared_error(y_test, result_lr)
# RMSE: raiz quadrada do MSE (facilita a visualização do valor).
rmse_lr = (np.sqrt(mse_lr))
# A métrica score varia entre 0 e 1, e indica percentualmente o quanto o modelo consegue explicar os valores observados.
# Quanto maior o valor mais explicativo é o modelo.
score_lr = model_lr.score(X_test, y_test)
print('Resultados do erro médio e score (acurácia do modelo) para o dataset de teste:\n  MSE: {}'.format(mse_lr) 
      + '\n RMSE: {}'.format(rmse_lr) 
      + '\nScore: {:2.3f}'.format(score_lr * 100) + '%')

###### 02. Árvore de Decisão

In [ ]:
tree = DecisionTreeRegressor()

In [ ]:
model_tree = tree.fit(X_train, y_train)
result_tree = tree.predict(X_test)

In [ ]:
mse_tree = mean_squared_error(y_test, result_tree)
rmse_tree = (np.sqrt(mse_tree))
score_tree = model_tree.score(X_test, y_test)
print('Resultados do erro médio e score (acurácia do modelo) para o dataset de treino:\n  MSE: {}'.format(mse_tree) 
      + '\n RMSE: {}'.format(rmse_tree) 
      + '\nScore: {:2.3f}'.format(score_tree * 100) + '%')

##### 03. Random Forest

In [ ]:
tree_rf = RandomForestRegressor()

In [ ]:
model_rf = tree_rf.fit(X_train, y_train)
result_rf = model_rf.predict(X_test)

In [ ]:
mse_rf = mean_squared_error(y_test, result_rf)
rmse_rf = (np.sqrt(mse_rf))
score_rf = model_rf.score(X_test, y_test)
print('Resultados do erro médio e score (acurácia do modelo) para o dataset de treino:\n  MSE: {}'.format(mse_rf) 
      + '\n RMSE: {}'.format(rmse_rf) 
      + '\nScore: {:2.3f}'.format(score_rf * 100) + '%')

##### 04. Random Forest com ajustes nos hiperparâmetros

In [ ]:
param = {'min_samples_leaf':[1,10],'min_samples_split':[2,10],'n_estimators':[100,250,500,750]}
rf = RandomForestRegressor()
grid = GridSearchCV(rf, param)

In [ ]:
# Executando o grid
grid.fit(X_train, y_train)

In [ ]:
# Verificando os melhores parâmetros
grid.best_params_

In [ ]:
rf_best = grid.best_estimator_

In [ ]:
result_grid = rf_best.predict(X_test)

In [ ]:
mse_grid = mean_squared_error(y_test, result_grid)
rmse_grid = (np.sqrt(mse_grid))
score_grid = rf_best.score(X_test, y_test)
print('Resultados do erro médio e score (acurácia do modelo) para o dataset de treino:\n  MSE: {}'.format(mse_grid) 
      + '\n RMSE: {}'.format(rmse_grid) 
      + '\nScore: {:2.3f}'.format(score_grid * 100) + '%')

##### Comparando os scores

In [ ]:
scores = [score_lr, score_tree, score_rf, score_grid]
labels = ['LinearRegression', 'DecisionTree', 'RandomForest', 'GridSearch']
plt.figure(figsize=(12,8))
plt.bar(labels, scores)

plt.show()